In [ ]:
import random, os
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import imageio.v3 as imageio
from IPython.display import clear_output
from render.CollectFrames import collect_saved_frames
matplotlib.use('Agg')
random.seed(42)

#### Function to generate forrest grid with specified density

In [12]:
def generate_forrest(height: int, width: int, density: float, burn_capacity: int):
    forrest_map = np.zeros((height, width))
    for y in range(height):
        for x in range(width):
            if random.uniform(0, 1) <= density:
                forrest_map[y][x] = burn_capacity
    return forrest_map

#### Function to execute the cellular automata for given number of iterations

In [13]:
def iterate_forrest_fire(
        forrest_map: np.ndarray,
        max_iter: int,
        burn_capacity: int,
        regrow_chance: float,
        selfign_chance: float):
    height, width = forrest_map.shape
    current_map = forrest_map.copy()
    yield current_map, 0
    for iteration in range(1, max_iter + 1):
        next_map = current_map.copy()
        for y in range(height):
            for x in range(width):
                value = current_map[y][x]
                # Burned tree or dirt
                if value == 0:
                    # Regrow to full tree with specified chance
                    if random.uniform(0, 1) < regrow_chance:
                        next_map[y][x] = burn_capacity
                # Healthy tree
                elif value == burn_capacity:
                    # Neighbor burn capacity values
                    top = lambda: current_map[y - 1][x]
                    bot = lambda: current_map[y + 1][x]
                    left = lambda: current_map[y][x - 1]
                    right = lambda: current_map[y][x + 1]
                    # Whether neighbors are currently burning
                    topb = y > 0 and top() > 0 and top() < burn_capacity
                    botb = y + 1 < height and bot() > 0 and bot() < burn_capacity
                    leftb = x > 0 and left() > 0 and left() < burn_capacity
                    rightb = x + 1 < width and right() > 0 and right() < burn_capacity
                    # If any of neighbors is burning or with specified chance start burning
                    if (True in [topb, botb, leftb, rightb]) or random.uniform(0, 1) < selfign_chance:
                        next_map[y][x] = value - 1
                # Currently burning tree
                elif value < burn_capacity and value != 0:
                    # Decrease burn capacity for tree
                    next_map[y][x] = value - 1
        current_map = next_map
        yield current_map, iteration

#### Generate forrest grid for the cellular automata

In [14]:
burn_capacity = 20
forrest_map = generate_forrest(125, 125, 0.5, burn_capacity)

#### Function to render cellular automata iteration

In [15]:
def render_forrest_iteration(
        forrest_map: np.ndarray, 
        iteration: int, 
        burn_capacity: int,
        scale = 4,
        fontsize = 14,
        title_fraction = 0.1,
        dpi = 100):
    height, width = forrest_map.shape
    rgb_map = np.zeros((height, width, 3), dtype=float)
    live_tree_mask = (forrest_map == burn_capacity)
    burning_mask = (forrest_map > 0) & (forrest_map < burn_capacity)
    rgb_map[live_tree_mask] = [0, 1, 0]
    if np.any(burning_mask):
        burning_values = forrest_map[burning_mask].astype(float)
        red_intensity = burning_values / burn_capacity
        rgb_map[burning_mask] = np.column_stack((
            red_intensity, 
            np.zeros_like(red_intensity), 
            np.zeros_like(red_intensity)
        ))
    map_pixel_width = int(width * scale)
    map_pixel_height = int(height * scale)
    title_fraction = max(0.01, min(0.9, title_fraction))
    total_pixel_height = int(map_pixel_height / (1.0 - title_fraction))
    total_pixel_width = map_pixel_width
    fig = plt.figure(figsize=(total_pixel_width / dpi, total_pixel_height / dpi), dpi=dpi)
    axes_bottom = 0
    axes_height = 1.0 - title_fraction
    ax = fig.add_axes([0, axes_bottom, 1, axes_height])
    title_string = f"Forrest fire cellular automata iteration: {iteration}"
    title_y_position = axes_bottom + axes_height + (title_fraction / 2.0)
    fig.text(0.5, title_y_position, title_string, ha='center', va='center', fontsize=fontsize)
    ax.imshow(rgb_map, interpolation='nearest', aspect='auto')
    ax.axis('off')
    fig.canvas.draw()
    frame = np.array(fig.canvas.buffer_rgba())[:, :, :3]
    plt.close(fig)
    return frame

#### Iterate the forrest fire cellular automata and collect frames

In [ ]:
folder_path = "gifs/fire/"
os.makedirs(folder_path, exist_ok=True)

In [18]:
for np_frame, iteration in iterate_forrest_fire(forrest_map, 10000, burn_capacity, 0.005, 0.001):
    print(f"FF iteration: {iteration}")
    frame = render_forrest_iteration(np_frame, iteration, burn_capacity)
    imageio.imwrite(os.path.join(folder_path, f"MazeQlearning{iteration:08d}.png"), frame)
clear_output()

In [19]:
gif_filename = "ForestFireAutomata.gif"
collect_saved_frames(folder_path, gif_filename, 15)

#### Visualize the forrsest fire cellular automata over time

![Forest Fire Automata](gifs/fire/ForestFireAutomata.gif)